In [1]:
import numpy as np
import pandas as pd 
import math
import random

In [2]:
def input_scp(file_path, n, m):
    with open(file_path, 'r') as file:
        line = file.readline().strip().split()
        n, m = map(int, line)
        
        costs = []
        while len(costs) < m:
            line1 = list(map(int, file.readline().strip().split()))
            costs += line1
        
        U = list(range(n))
        U_S = [[] for _ in range(n)]
        for i in range(n):
            nn = int(file.readline())
            U_S_i = []
            while len(U_S_i) < nn:
                linee = list(map(int, file.readline().strip().split()))
                U_S_i += linee
            U_S[i] = U_S_i
        
        S = [None]*m
        for i in range(m):
            S[i] = []
        for i in range(n):
            for j in U_S[i]:
                S[j-1].append(i)
        return costs, U, S


In [15]:
def prob_analysis(pheromone, sets, set_costs, unseen_elem, alpha, beta):
    prob = 0.0
    probabilities = np.zeros(len(sets))
    for i, j in enumerate(sets):
        prob = 0.0

        for element in j:
            if element in unseen_elem:
                prob += pheromone[i][element - 1] ** alpha

        if prob == 0:
            probabilities[i] = 0
        else:
            probabilities[i] = prob / (set_costs[i] ** beta)

    probabilities /= np.sum(probabilities)

    return probabilities

The given code defines a function input_scp that reads and processes input data from a file specified by file_path for the set cover problem. The function expects two additional parameters, n and m, representing the number of elements and sets, respectively.

Inside the function, the file is opened, and the first line is read and split into two integers, n and m, overriding the function parameters. The subsequent lines are then read to populate the costs list with m costs.

The lists U and U_S are initialized. U contains elements from 0 to n-1, and U_S is a list of empty lists, with n empty lists in total. The next n lines are read to populate the U_S list. Each line begins with an integer nn, representing the number of elements in a set, followed by nn integers that represent the elements themselves. These elements are added to the corresponding inner list in U_S.

A list S is created with m elements initially set to None. Then, the nested loops iterate over each i in the range of n, and for each element j in U_S[i], the corresponding S element at index j-1 (adjusting for 0-based indexing) is accessed, and i is appended to it.

Finally, the function returns the costs, U, and S lists, providing the processed input data for the set cover problem.

In [4]:
file_path = '/content/scp41.txt' 
costs, U, S = input_scp(file_path, 200, 1000)

print(len(costs), "Costs:", costs)
print(len(U), "U:", U)
print(len(S), "S:", S)

1000 Costs: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 

In [16]:
def modify_set_cover(number_of_ants, iterations, evaporation_rate, alpha, beta, universe, sets, set_costs):
    best_solution = None
    minCost = 99999999
    pheromone = np.ones((len(sets), len(universe)))
    for _ in range(iterations):
        solutions = []
        costs = []

        for ii in range(number_of_ants):
            cost = 0
            unseen_elem = set(universe)
            solution = []
            while unseen_elem:
                probabilities = prob_analysis(pheromone, sets, set_costs, unseen_elem, alpha, beta)
                set_idx = np.random.choice(len(probabilities), p=probabilities)
                solution.append(set_idx)
                unseen_elem -= set(sets[set_idx])
                cost += set_costs[set_idx]
            solutions.append(solution)
            costs.append(cost)
            if cost < minCost:
                best_solution = solution #saving the best solution
                minCost = cost          # saving the best cost
            for set_idx in solution:
                pheromone[set_idx] += 1.0
        pheromone *= (1.0 - evaporation_rate)
    return best_solution, minCost




In [14]:
universe = U
sets = S
set_costs = costs 
number_of_ants = 10
iterations = 100
evaporation_rate = 0.5
alpha = 1.0
beta = 1.0

best_solution, best_cost = modify_set_cover(number_of_ants, iterations, evaporation_rate, alpha, beta, universe, sets, set_costs)

print("Best Solution:", best_solution)
print("Cost:", best_cost)

Best Solution: [158, 12, 16, 37, 34, 0, 6, 15, 3, 44, 1, 5, 70, 117, 120, 14, 11, 58, 21, 56, 23, 7, 52, 30, 320, 8, 9, 17, 217, 73, 139, 2, 4, 22, 115, 147, 156, 129, 19, 215, 20, 27, 45, 74, 91, 38, 68, 138, 342, 29, 98, 54, 24, 55, 106, 61, 123, 268, 251, 35, 103, 101, 90, 229, 122, 180]
Cost: 330


In [17]:
file_path = '/content/scp52.txt' 
costs, U, S = input_scp(file_path, 200, 2000)

print(len(costs), "Costs:", costs)
print(len(U), "U:", U)
print(len(S), "S:", S)

2000 Costs: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 

In [18]:
universe = U
sets = S
set_costs = costs 
number_of_ants = 10
iterations = 100
evaporation_rate = 0.5
alpha = 1.0
beta = 1.0

best_solution, best_cost = modify_set_cover(number_of_ants, iterations, evaporation_rate, alpha, beta, universe, sets, set_costs)

print("Best Solution:", best_solution)
print("Cost:", best_cost)

Best Solution: [6, 38, 3, 0, 16, 265, 37, 2, 35, 23, 8, 7, 90, 4, 120, 11, 268, 106, 158, 68, 118, 14, 91, 72, 177, 92, 73, 55, 19, 12, 5, 217, 22, 34, 24, 70, 17, 20, 108, 251, 61, 58, 30, 45, 27, 139, 29, 162, 33, 101, 15, 21, 1, 324, 156, 44, 46, 115, 138, 151, 142, 131, 218, 112, 229]
Cost: 330


In [40]:
file_path = '/content/scp62.txt' 
costs, U, S = input_scp(file_path, 200, 1000)

print(len(costs), "Costs:", costs)
print(len(U), "U:", U)
print(len(S), "S:", S)

1000 Costs: [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2

In [41]:
universe = U
sets = S
set_costs = costs 
number_of_ants = 10
iterations = 100
evaporation_rate = 0.5
alpha = 1.0
beta = 1.0

best_solution, best_cost = modify_set_cover(number_of_ants, iterations, evaporation_rate, alpha, beta, universe, sets, set_costs)

print("Best Solution:", best_solution)
print("Cost:", best_cost)

Best Solution: [79, 18, 3, 5, 2, 39, 6, 102, 10, 57, 47, 24, 20, 25, 11, 9, 30, 16, 106, 17, 67, 21, 0, 28, 1, 23, 58, 27, 60, 37, 26, 82, 63, 15, 44, 76, 32, 53, 77]
Cost: 165
